Problem Statement   Cryptocurrency markets are highly volatile, and liquidity plays a crucial role in market stability. Liquidity refers to the ease with which assets can be bought or sold without significantly impacting the price. A lack of liquidity can lead to increased price fluctuations and market instabil

Ans:1. Data Collection
Since I can't download directly from your Google Drive here, I will assume you will download the dataset manually.

Suppose you have downloaded it — say a file like crypto_data.csv.

In Python:

In [ ]:
import pandas as pd

# Load the dataset
df = pd.read_csv('crypto_data.csv')
print(df.head())


. Data Preprocessing
Handling missing values:

In [ ]:
# Check missing values
print(df.isnull().sum())

# Fill missing values with forward fill or interpolation
df.fillna(method='ffill', inplace=True)


Normalizing/Scaling numerical columns (volume, price, etc.):

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(df[['open', 'high', 'low', 'close', 'volume']])
scaled_df = pd.DataFrame(scaled_features, columns=['open', 'high', 'low', 'close', 'volume'])


 Exploratory Data Analysis (EDA)
Summary statistics:

In [ ]:
print(df.describe())


Visualizations:

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Line plot of closing price
plt.figure(figsize=(10,5))
plt.plot(df['date'], df['close'])
plt.title('Closing Price Over Time')
plt.show()

# Heatmap
plt.figure(figsize=(10,8))
sns.heatmap(df.corr(), annot=True)
plt.show()


 Feature Engineering
Create new features important for liquidity prediction:

Moving Averages:

In [ ]:
df['volume_ma7'] = df['volume'].rolling(window=7).mean()
df['volume_ma30'] = df['volume'].rolling(window=30).mean()


Volatility:

python
Copy
Edit


In [ ]:
df['price_volatility'] = df['close'].rolling(window=7).std()


Liquidity Ratio (example: volume divided by close price):

In [ ]:
df['liquidity_ratio'] = df['volume'] / df['close']


5. Model Selection
Since you are predicting a continuous value (liquidity levels), this is a regression task.

Recommended models:

Random Forest Regressor

XGBoost Regressor

Example:

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Define features and target
features = ['open', 'high', 'low', 'close', 'volume', 'volume_ma7', 'volume_ma30', 'price_volatility', 'liquidity_ratio']
target = 'liquidity_ratio'  # or any defined liquidity target you choose

X = df[features]
y = df[target]

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model
model = RandomForestRegressor()
model.fit(X_train, y_train)

# Predict
y_pred = model.predict(X_test)

# Evaluate
print("RMSE:", mean_squared_error(y_test, y_pred, squared=False))
print("MAE:", mean_absolute_error(y_test, y_pred))
print("R² Score:", r2_score(y_test, y_pred))


6. Hyperparameter Tuning (Optional for better results)
Use GridSearchCV or RandomizedSearchCV for tuning.



7. Model Testing & Validation
Check how well the model predicts unseen data.

Plot actual vs. predicted:

In [ ]:
plt.figure(figsize=(10,5))
plt.plot(y_test.values, label='Actual')
plt.plot(y_pred, label='Predicted')
plt.legend()
plt.title('Actual vs Predicted Liquidity Levels')
plt.show()


8. Deployment (Optional)
You can deploy this model using Flask or Streamlit.

Simple Streamlit code:

In [ ]:
import streamlit as st
import pickle

model = pickle.load(open('model.pkl', 'rb'))

st.title('Cryptocurrency Liquidity Predictor')

open_price = st.number_input('Open Price')
high_price = st.number_input('High Price')
low_price = st.number_input('Low Price')
close_price = st.number_input('Close Price')
volume = st.number_input('Volume')

if st.button('Predict Liquidity'):
    input_data = [[open_price, high_price, low_price, close_price, volume]]
    prediction = model.predict(input_data)
    st.success(f'Predicted Liquidity Level: {prediction[0]:.4f}')
